In [3]:
pip install stringio


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement stringio (from versions: none)
ERROR: No matching distribution found for stringio


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

contest_name = input("Enter the Name of Contest").replace(" ", "-")
# contest_name = "the-great-sqlizer"

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Enable headless mode for background execution

# Set the path to the ChromeDriver
driver = webdriver.Chrome(options=chrome_options)

# Open the CSV file and prepare to write to it
with open(f'{contest_name}.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Username', 'Score', 'Questions Solved', 'Time'])

    
    # Start from the first page
    page = 1
    try:
        while True:
            url = f"https://www.hackerearth.com/challenges/competitive/{contest_name}/leaderboard/page/{page}/"
            driver.get(url)
            driver.maximize_window()

            # Wait for the table to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.align-top"))
            )

            # Extract rows from the leaderboard table
            rows = driver.find_elements(By.CSS_SELECTOR, "tbody.align-top tr")
            if not rows:
                break  # If no rows are found, stop the loop

            for row in rows:
                rank = row.find_element(By.CSS_SELECTOR, "span.float-left.medium-margin-right.weight-700.dark").text.strip().replace(".","")
                name = row.find_element(By.CSS_SELECTOR, "div.no-color.hover-link.weight-600").text.strip()
                username = row.find_element(By.CSS_SELECTOR, "div.gray-text.body-font-small.hover-link").text.strip()
                score_data = row.find_element(By.CSS_SELECTOR, "td.align-center").text.strip()
                time_data = row.find_elements(By.CSS_SELECTOR, "td.align-center")[1].text.strip()

                score, questions_solved = score_data.split()
                questions_solved = questions_solved.strip('()')

                writer.writerow([rank, name, username, score, questions_solved, time_data])

            # Check for the next page button and whether it is clickable
            next_button = driver.find_elements(By.CSS_SELECTOR, "i.fa.fa-angle-right.dark")
            if next_button and next_button[0].is_displayed():  # Check if it is displayed and presumably clickable
                next_button[0].click()
                page += 1
            else:
                break  # No more pages

    finally:
        driver.quit()
